# **Projeto Boston Housing**

Neste projeto você deverá desenvolver um projeto offline de ponta a ponta. Nesta atividade, porém, vamos trabalhar com o Boston Housing.
Você pode fazer em dupla, mas todos os membros devem participar de todas as etapas, inclusive da gravação do vídeo (ver etapa 3). Todos os membros da equipe devem fazer submissão individual no moodle, ainda que o trabalho seja feito em equipe.

As etapas do projeto são as seguintes:

Desenvolver o projeto completamente, o que consiste:

1. Descrição do conjunto de dados;

2. Separação do conjunto em treino e teste;

3. Visualização do conjunto de dados (análise exploratória básica);

4. Preparação do conjunto de dados;

5. Comparar ao menos 3 modelos de machine learning e algumas configuração de hiperparâmetros, justificando a escolha do melhor modelo;

6. Você deve ainda justificar a escolha da métrica utilizada;

7. Deve discutir a técnica utilizada para validar o modelo e deve explicar como que o seu modelo evita o "snooping bias/data leakage";

8. Fazer teste final para obter um erro aproximado.

2 - A segunda etapa consiste em você disponibilizar o projeto no GitHub, através de um link. Você não precisa se aprofundar em GitHub, é suficiente você criar uma conta e fazer upload manualmente pelo browser.  A ideia é que você se acostume a criar um portfólio para as suas atividades sorriso

3 - A terceira etapa consite em você gravar um vídeo de, no máximo, 15 min (20 min se feito em dupla). Neste vídeo você fará uma apresentação dos principais pontos do projeto, ilustrando quais foram as dificuldades. Precisa ficar claro que você domina todas as etapas de um projeto de ML offline. Minha sugestão é fazer upload em um link privado do YouTube, se ficar bom você pode deixar público para o portfólio de vocês.

4 - O formato da entrega é jupyter notebook para o projeto, lá você pode adicionar o link para o vídeo também.

Você pode encontrar as informações e o dataset aqui nesse link.

Dica: Revise aulas sobre Califórnia Housing, assim como o notebook disponível no meu github.

# 0. Importa Dados Boston Housing

Fonte: https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

Existem 14 atributos em cada caso do conjunto de dados. Eles são:

CRIM - taxa de criminalidade per capita por cidade

ZN - proporção de terrenos residenciais zoneados para lotes acima de 25.000 pés quadrados

INDUS - proporção de acres de negócios não varejistas por cidade.

CHAS - Variável fictícia do Rio Charles (1 se o trato limita o rio; 0 caso contrário)

NOX - concentração de óxidos nítricos (partes por 10 milhões)

RM - número médio de cômodos por moradia

AGE - proporção de unidades ocupadas pelo proprietário construídas antes de 1940

DIS - distâncias ponderadas para cinco centros de emprego de Boston

RAD - índice de acessibilidade a rodovias radiais

TAX - taxa de imposto sobre a propriedade com valor integral por US$ 10.000
PTRATIO - proporção aluno-professor por cidade

B - 1000(Bk - 0,63)^2 onde Bk é a proporção de negros por cidade

LSTAT - % de status inferior da população

MEDV - Valor mediano de casas ocupadas pelo proprietário em US$ 1.000

In [ ]:
#### INSTALAR isso =)
!pip install dash

In [ ]:
#### Replicabilidade das Coisas
seed = 26011994

In [ ]:
#### Libs
import pandas as pd

In [ ]:
import pandas as pd

# URL do dataset no UCI Machine Learning Repository
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data"

# Definindo os nomes das colunas (segundo a documentação do dataset no UCI)
column_names = [
    "CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS",
    "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"
]

# Carregando o dataset
housing = pd.read_csv(url, delim_whitespace=True, names=column_names)

# Exibindo as primeiras linhas
print(housing.head())

### ** Note que existem variáveis inteiras as quais não necessáriamente representam quantidades. **

# 1. Descrição do conjunto de dados

In [ ]:
housing.info()

In [ ]:
round(housing.describe(),2)

In [ ]:
### Verifica Missing

# Calcular o percentual de valores ausentes por variável
missing_percentage = housing.isnull().mean() * 100

# Criar um DataFrame para exibir de forma mais clara
missing_df = missing_percentage.reset_index()
missing_df.columns = ['Variável', 'Percentual de Missing']
missing_df = missing_df.sort_values(by='Percentual de Missing', ascending=False)

# Exibir o resultado
print(missing_df)


In [ ]:
# @title CHAS - Variável fictícia do Rio Charles (1 se o trato limita o rio; 0 caso contrário)
housing.loc[:,['CHAS']].value_counts()

In [ ]:
# @title Dashboard de Análise Descritiva
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go

df = housing
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard de Análise Descritiva"),

    # Dropdown para selecionar a variável
    html.Label("Selecione a variável:"),
    dcc.Dropdown(
        id='dropdown-variable',
        options=[{'label': col, 'value': col} for col in df.columns],
        value='MEDV'
    ),

    # Saída das estatísticas descritivas
    html.Div(id='output-descriptive-stats'),

    # Gráfico de distribuição
    dcc.Graph(id='output-distribution-plot')
])

# Callback para atualizar as estatísticas descritivas e o gráfico de distribuição
@app.callback(
    [Output('output-descriptive-stats', 'children'),
     Output('output-distribution-plot', 'figure')],
    [Input('dropdown-variable', 'value')]
)
def update_output(variable):
    # Estatísticas descritivas
    desc_stats = df[variable].describe().to_frame().T
    desc_table = html.Table([
        html.Tr([html.Th(col) for col in desc_stats.columns]),
        html.Tr([html.Td(desc_stats[col].values[0]) for col in desc_stats.columns])
    ])

    # Gráfico de distribuição
    fig = px.histogram(df, x=variable, nbins=30, title=f'Distribuição de {variable}')
    fig.update_layout(bargap=0.2)

    return desc_table, fig

# Executa o app
if __name__ == '__main__':
    app.run_server(debug=True , port=8051)

In [ ]:
# @title Visão De Todas as Variáveis ao Mesmo Tempo
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

Podemos observar, intuitivamente, que nem todas as distribuições seguem uma distibuição normal.

In [ ]:
# @title Correlação Linear entre Variáveis
# Calcular a matriz de correlação
correlation_matrix = housing.corr()

# Criar o heatmap de correlação com valores sobrepostos
fig = px.imshow(correlation_matrix,
                text_auto=True,  # Mostra os valores da correlação no gráfico
                color_continuous_scale="RdBu_r",
                aspect="auto",
                title="Mapa de Correlação entre Variáveis")

# Exibir o gráfico
fig.show()

Aqui vale ressaltar que o a variável CHAS é uma representação numérica aleatória de categorias sem relação de ordem. Assim, a análise de correção perde um pouco do sentido.

Será interessante olhar outras méticas mais fortes que nos diga se de fato ela será poderosa em nosso modelo, ou não.

In [ ]:
# @title ### **Vamos analisar se existe algum padrão aparente entre as variáveis coletadas e a variável alvo (MEDV)**
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np

df = housing
# Inicialização do app Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard de Correlação e Regressão com MEDV"),

    # Dropdown para selecionar a variável no eixo X
    html.Label("Selecione a variável (eixo X):"),
    dcc.Dropdown(
        id='dropdown-variable',
        options=[{'label': col, 'value': col} for col in df.columns if col != 'MEDV'],
        value='LSTAT'
    ),

    # Gráfico de dispersão
    dcc.Graph(id='scatter-plot')
])

# Callback para atualizar o gráfico de dispersão
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('dropdown-variable', 'value')]
)
def update_scatter(variable):
    # Calcular o coeficiente de correlação
    corr_coef = np.corrcoef(df[variable], df['MEDV'])[0, 1]

    # Criar o gráfico de dispersão com linha de regressão
    fig = px.scatter(df, x=variable, y="MEDV", trendline="ols",
                     title=f'Dispersão de MEDV vs {variable} (Coeficiente de Correlação = {corr_coef:.2f})')

    # Adicionar coeficiente de correlação no título
    fig.update_layout(title=dict(x=0.5))  # Centraliza o título
    fig.update_traces(marker=dict(size=6))

    return fig

# Executa o app
if __name__ == '__main__':
    app.run_server(debug=True, port=8052)

Aqui observamos mais de perto o comportamento dos dados em relação a variável target.

Na análise passada vimos quais tem maior correlação. Como a correlação linear usual é o coeficiente angular da regressão linear entre as duas variáveis, aproveitei para plotar as regresões as  originaram, a fim de observar se estão fazendo sentido.

Um **grande perigo** é que podem ocorrer casos em que exista uma var x1 = x2^2. Nesse caso, o coeficente de correlção **linear** será zero, mesmo uma sendo escrita completamente em função da outra.


In [ ]:
# @title  Conforme segue o exemplo abaixo.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, linregress

# Gerando dados
np.random.seed(0)
x1 = np.linspace(-10, 10, 100)
x2 = x1**2  # x2 é uma função não-linear de x1

# Coeficiente de correlação linear
correlation, _ = pearsonr(x1, x2)
print(f"Coeficiente de correlação linear entre x1 e x2: {correlation:.2f}")

# Regressão linear
slope, intercept, r_value, p_value, std_err = linregress(x1, x2)
line = slope * x1 + intercept  # Linha de regressão linear

# Plotando
plt.figure(figsize=(10, 6))
sns.scatterplot(x=x1, y=x2, label="Dados")
plt.plot(x1, line, color="red", linestyle="--", label="Regressão Linear")
plt.title(f"Gráfico de x2 = x1^2 com Coeficiente de Correlação Linear = {correlation:.2f}")
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend()
plt.show()


# **2. Separação do conjunto em treino e teste**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Função para discretizar variáveis numéricas e estratificar
def stratified_sampling(data, max_bins, columns):
    df = data.copy()

    # Discretizar colunas numéricas
    for column in columns:
        unique_values = df[column].nunique()
        if unique_values > max_bins:
            bins = max_bins
            df[column + '_decile'] = pd.qcut(df[column], q=bins, labels=[str(i+1) for i in range(bins)], duplicates='drop')
        else:
            bins = unique_values
            df[column + '_decile'] = pd.cut(df[column], bins=bins, labels=[str(i+1) for i in range(bins)], include_lowest=True)

    # Criar coluna de estratificação
    df['stratify_column'] = df[[col + '_decile' for col in columns]].apply(lambda row: '_'.join(row.astype(str)), axis=1)

    # Ajustar classes com menos de 2 observações
    class_counts = df['stratify_column'].value_counts()
    rare_classes = class_counts[class_counts < 2].index
    if len(rare_classes) > 0:
        df.loc[df['stratify_column'].isin(rare_classes), 'stratify_column'] = 'OTHER'

    # Realizar amostragem estratificada
    train, test = train_test_split(df, stratify=df['stratify_column'], test_size=0.25, random_state=42)

    # Remover colunas auxiliares
    columns_to_drop = [col + '_decile' for col in columns] + ['stratify_column']
    train.drop(columns=columns_to_drop, inplace=True)
    test.drop(columns=columns_to_drop, inplace=True)

    return train, test

# Aplicar amostragem estratificada
train_set, test_set = stratified_sampling(
    housing,
    25,
    ['RM', 'DIS', 'LSTAT', 'MEDV']
)

###Verificar tamanhos
print(f"Train size: {len(train_set)}")
print(f"Test size: {len(test_set)}")
print(f"Total: {len(train_set) + len(test_set)}")
print('Train: '+ str(round(train_set.shape[0]/housing.shape[0], 4)*100) +'%')
print('Test: ' + str(round(test_set.shape[0] /housing.shape[0], 4)*100) +'%')

Vamos criar uma amostragem aleátoria para fins de comparação

In [ ]:
train_set_aleatorio, test_set_aleatorio = train_test_split(housing, test_size=0.2, random_state=seed)
print('Train: '+ str(round(train_set_aleatorio.shape[0]/housing.shape[0], 4)*100) +'%')
print('Test: ' + str(round(test_set_aleatorio.shape[0] /housing.shape[0], 4)*100) +'%')

In [ ]:
import matplotlib.pyplot as plt

def plot_train_test_distributions_percentual(housing, train, test,  columns):
    plt.figure(figsize=(24, 18))  # Ajuste do tamanho para comportar mais informações
    for i, column in enumerate(columns, 1):
        plt.subplot(5, 4, i)  # Ajusta a grade para 5x4, caso tenha mais colunas
        housing[column].hist(bins=50, alpha=0.5, label='Housing', color='purple', density=True, linewidth=0.7, edgecolor='black')
        train[column].hist(bins=50, alpha=0.5, label='Train Stratified', color='blue', density=True, linewidth=0.7, edgecolor='black')
        test[column].hist(bins=50, alpha=0.5, label='Test Stratified', color='orange', density=True, linewidth=0.7, edgecolor='black')

        plt.title(column, fontsize=12)
        plt.xlabel('Value', fontsize=10)
        plt.ylabel('Density', fontsize=10)
        plt.legend(loc='upper right', fontsize=8)
    plt.tight_layout()
    plt.show()

# Aplicar a função para visualizar distribuições percentuais
plot_train_test_distributions_percentual(
    housing, train_set, test_set, train_set.columns
)


In [ ]:
import matplotlib.pyplot as plt

def plot_variable_bins_with_labels(housing, columns, bins=10):
    plt.figure(figsize=(20, 15))
    for i, column in enumerate(columns, 1):
        plt.subplot(4, 4, i)

        # Criar bins com descrição dos intervalos
        housing_bins = pd.cut(housing[column], bins=bins, include_lowest=True)
        bin_counts = housing_bins.value_counts(normalize=True).sort_index()

        # Extrair labels descritivos dos intervalos
        bin_labels = [f"{interval.left:.2f} - {interval.right:.2f}" for interval in bin_counts.index]

        # Plotar distribuição percentual
        plt.bar(bin_labels, bin_counts.values, color='blue', alpha=0.7)
        plt.title(f'{column} - Distribution')
        plt.xlabel('Intervals')
        plt.ylabel('Percentage')
        plt.xticks(rotation=45, ha='right')  # Rotacionar rótulos para melhor visualização

    plt.tight_layout()
    plt.show()

# Plotar distribuições de todas as variáveis do conjunto housing
plot_variable_bins_with_labels(housing, housing.columns)


In [ ]:
# @title Verifica Eficiência da Amostragem Estratificada
import pandas as pd
import numpy as np

def variable_bins_dist(df_referencia_tmp, df_tmp, columns, bins=20):
    """
    Aplica bins consistentes baseados no dataframe de referência a outro dataframe
    e retorna a distribuição normalizada das variáveis binned.

    Args:
    - df_referencia: DataFrame de referência para calcular os bins.
    - df: DataFrame ao qual os bins serão aplicados.
    - columns: Lista de colunas para aplicar o binning.
    - bins: Número de bins a criar.

    Returns:
    - bin_counts: DataFrame com as distribuições normalizadas das variáveis.
    """
    df_referencia = df_referencia_tmp.copy()
    df = df_tmp.copy()

    bin_counts = pd.DataFrame()

    # Criar bins com base no df_referencia
    bin_edges = {}
    for column in columns:
        # Verificar se a coluna contém valores numéricos
        if not pd.api.types.is_numeric_dtype(df_referencia[column]):
            raise ValueError(f"A coluna {column} não é numérica. Verifique os dados.")

        # Remover NaNs antes de calcular os limites
        col_data = df_referencia[column].dropna()

        # Obter os limites dos intervalos dos bins
        _, bin_edges[column] = pd.cut(col_data, bins=bins, retbins=True, include_lowest=True)

    # Aplicar os bins ao df e calcular a distribuição
    for column in columns:
        # Substituir NaNs por um valor para evitar erros
        df[column] = df[column].fillna(df[column].median())

        # Aplicar os bins usando os limites calculados
        df['binned_' + column] = pd.cut(df[column], bins=bin_edges[column], include_lowest=True)
        counts = df['binned_' + column].value_counts(normalize=True).sort_index().reset_index()

        # Adicionar os nomes corretos às colunas
        counts.columns = ['Ranks', 'Perc_Distribuicao']
        counts['Variavel'] = column

        # Reorganizar as colunas para manter a ordem
        counts = counts[['Variavel', 'Ranks', 'Perc_Distribuicao']]

        # Concatenar com o DataFrame de resultados
        bin_counts = pd.concat([bin_counts, counts], ignore_index=True)

    return bin_counts

# Exemplo de uso:
# Certifique-se de que `columns` contém apenas colunas numéricas.
columns = housing.select_dtypes(include=[np.number]).columns

# Calcular distribuições no dataset de referência
dist_housing = variable_bins_dist(housing, housing, columns, bins=10)

# Aplicar os mesmos bins no test_set e no test_set_aleatorio
dist_test_set = variable_bins_dist(housing, test_set, columns, bins=10)
dist_test_set_aleatorio = variable_bins_dist(housing, test_set_aleatorio, columns, bins=10)

# Adicionar uma coluna de origem para identificar os DataFrames
dist_housing['Origem'] = 'housing'
dist_test_set['Origem'] = 'test_set'
dist_test_set_aleatorio['Origem'] = 'test_set_aleatorio'

# Concatenar os DataFrames verticalmente
final_df = pd.concat([dist_housing, dist_test_set, dist_test_set_aleatorio], axis=0, ignore_index=True)

# Transpor o DataFrame final para organizar as distribuições por variável e rank
final_df_pivot = (
    final_df.pivot_table(
        index=['Variavel', 'Ranks'],  # Índices para o pivot (join será por estes)
        columns='Origem',            # Coluna para criar colunas separadas
        values='Perc_Distribuicao'   # Valores a serem preenchidos
    )
    .reset_index()                   # Resetar índice após o pivot
)

# Renomear colunas para deixar os nomes mais descritivos
final_df_pivot.columns = [
    'Variavel', 'Ranks',
    'Perc_Distribuicao_Housing',
    'Perc_Distribuicao_Test_Set',
    'Perc_Distribuicao_Test_Set_Aleatorio'
]

# Preencher valores NaN com 0 (se necessário)
final_df_pivot = final_df_pivot.fillna(0)

# Calcular a diferença absoluta em relação ao Housing
final_df_pivot['Diff_Abs_Test_Set'] = (
    abs(final_df_pivot['Perc_Distribuicao_Housing'] - final_df_pivot['Perc_Distribuicao_Test_Set'])
)

final_df_pivot['Diff_Abs_Test_Set_Aleatorio'] = (
    abs(final_df_pivot['Perc_Distribuicao_Housing'] - final_df_pivot['Perc_Distribuicao_Test_Set_Aleatorio'])
)

final_df_pivot

# Agrupar por variável e calcular a média das diferenças absolutas
summary_df = (
    final_df_pivot.groupby('Variavel')[['Diff_Abs_Test_Set', 'Diff_Abs_Test_Set_Aleatorio']]
    .mean()
    .reset_index()
)

# Ordenar do maior para o menor com base na diferença absoluta média
summary_df = summary_df.sort_values(by=['Diff_Abs_Test_Set', 'Diff_Abs_Test_Set_Aleatorio'], ascending=False)

# Renomear colunas para deixá-las mais descritivas
summary_df.columns = ['Variavel', 'Media_Diff_Abs_Test_Set', 'Media_Diff_Abs_Test_Set_Aleatorio']
summary_df

#**3. Visualização do conjunto de dados (análise exploratória básica)**

In [ ]:
train_set.plot(kind="scatter", x="LSTAT", y="RM",
    s=train_set["CRIM"], label="Taxa de Criminalidade", figsize=(10,7),
    c="MEDV", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False) #sharex=false é só pra corrigir um bug de display https://github.com/pandas-dev/pandas/issues/10611
plt.legend()

In [ ]:
corr_matrix = housing.corr()
corr_matrix["MEDV"].sort_values(ascending=False)
corr_matrix

from pandas.plotting import scatter_matrix
scatter_matrix(housing, figsize=(12, 8))

#**4. Preparação do conjunto de dados**

Aqui evitamos o snooping bias estudando o dataframe de train.

In [ ]:
housing = train_set.drop("MEDV", axis=1).copy() # O método drop cria cópia sem a coluna em questao
housing_labels = train_set["MEDV"].copy() #salvando uma cópia

In [ ]:
### Verifica Missing

# Calcular o percentual de valores ausentes por variável
missing_percentage = housing.isnull().mean() * 100

# Criar um DataFrame para exibir de forma mais clara
missing_df = missing_percentage.reset_index()
missing_df.columns = ['Variável', 'Percentual de Missing']
missing_df = missing_df.sort_values(by='Percentual de Missing', ascending=False)

# Exibir o resultado
print(missing_df)


In [ ]:
from sklearn.impute import SimpleImputer  # Para lidar com valores ausentes
from sklearn.preprocessing import StandardScaler  # Para normalizar/padronizar os dados
from sklearn.pipeline import Pipeline  # Para criar pipelines
from sklearn.preprocessing import FunctionTransformer  # Para transformações personalizadas

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing.select_dtypes(include=[np.number]))
housing_num_tr

In [ ]:
from sklearn.compose import ColumnTransformer  # Para transformar colunas específicas
from sklearn.preprocessing import OneHotEncoder

num_attribs = list(housing.select_dtypes(include=[np.number]))
cat_attribs = list(housing.select_dtypes(include=[object]))

#Este é o pipeline completo!
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs), #um pipeline dentro do outro
        ("cat", OneHotEncoder(), cat_attribs),
    ])

""" Lembrando: num_pipeline é o pipeline que transforma variavéis numéricas

num_pipeline = Pipeline([
      ('imputer', SimpleImputer(strategy="median")),
      ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
      ('std_scaler', StandardScaler()),
    ])
"""

#### Salva conjunto Preparado
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

#**5. Comparar ao menos 3 modelos de machine learning e algumas configuração de hiperparâmetros, justificando a escolha do melhor modelo**

In [ ]:
### Regressao Linear
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)
#Ei Regressão linear, encontre os parâmetros que melhor aproxima os dados

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data) #Full pipeline

print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

In [ ]:
from sklearn.metrics import mean_squared_error as MSE

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = MSE(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse) #Não é necessariamente obrigatório
print(lin_rmse)
print(lin_mse)

In [ ]:
from sklearn.metrics import mean_absolute_error as MAE

lin_mae = MAE(housing_labels, housing_predictions)
lin_mae

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state= seed, min_weight_fraction_leaf = 0.01) ### Evita overfitting
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = MSE(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

##**Avaliação de Modelos**

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)

#cv = 10 é número de pedaços

tree_rmse_scores = np.sqrt(-scores)
print(tree_rmse_scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42, min_weight_fraction_leaf = 0.01)
forest_reg.fit(housing_prepared, housing_labels) #Treinar modelo

housing_predictions = forest_reg.predict(housing_prepared) #Predizer
forest_mse = MSE(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
print(forest_rmse)

from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

#**Ajustando o Modelo Escolhido**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # Vamos tentar 12 = 3x4 combinação de parâmetros
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # Tentar 6 = 2×3 combinações do bootstrap no modo 'Falso'
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=seed, min_weight_fraction_leaf = 0.01)

# Vamos treinar com 5-folds, então temos (12+6)*5=90 rodadas de treinamento!!!

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
pd.DataFrame(grid_search.cv_results_)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=1000),
        'max_features': randint(low=1, high=11),
    }

forest_reg = RandomForestRegressor(random_state=seed,min_weight_fraction_leaf = 0.01)

rnd_search = RandomizedSearchCV(forest_reg,
                                param_distributions=param_distribs,
                                n_iter=100,
                                cv=5,
                                scoring='neg_mean_squared_error',
                                random_state=seed)

rnd_search.fit(housing_prepared, housing_labels)

In [ ]:
rnd_search.best_params_

In [ ]:
'''cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)'''

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

# Criar um DataFrame com as importâncias e os nomes das variáveis
feature_importance_df = pd.DataFrame({
    'Feature': housing.columns,  # Nomes das variáveis
    'Importance': feature_importances  # Importância de cada variável
})

feature_importance_df.sort_values(by='Importance', ascending=False)

#**Modelo e Teste Final**

In [ ]:
# 1. Instanciar o modelo
final_model_best_params = rnd_search.best_params_
final_model = RandomForestRegressor(random_state=seed
                                    , min_weight_fraction_leaf = 0.01
                                    , **final_model_best_params
                                    )

# 2. Treinar no conjunto de treino
final_model.fit(housing_prepared, housing_labels)

# 3. Prever num conjunto
X_test = test_set.drop("MEDV", axis=1).copy()
y_test = test_set["MEDV"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = MSE(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

print(final_rmse)
print(final_mse)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo do seaborn para o gráfico
sns.set(style="whitegrid")

# Criar um DataFrame para facilitar a manipulação dos dados
import pandas as pd
results_df = pd.DataFrame({
    "Real": y_test,
    "Predito": final_predictions
})

# Configurar o tamanho da figura
plt.figure(figsize=(8, 6))  # Tamanho menor

# Plotar as distribuições
sns.kdeplot(results_df["Real"], label="Real", color="blue", linewidth=1.5, fill=True, alpha=0.3)
sns.kdeplot(results_df["Predito"], label="Predito", color="orange", linewidth=1.5, fill=True, alpha=0.3)

# Títulos e legendas
plt.title("Distribuição de MEDV Real vs Predito (Teste)", fontsize=14)
plt.xlabel("MEDV", fontsize=10)
plt.ylabel("Densidade", fontsize=10)
plt.legend(fontsize=10)
plt.grid(visible=True, alpha=0.3)

# Mostrar o gráfico
plt.tight_layout()  # Melhor ajuste
plt.show()
